# 메인페이지에서의 친밀도 방 배치

참고) https://data-science-hi.tistory.com/82

In [61]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD

In [89]:
import sys

print(sys.version)

3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]


In [62]:
# 팔로잉DB, 팔로워DB 가져오기
Following_df=pd.read_excel("Following.xlsx")
Follower_df=pd.read_excel("Follower.xlsx")
Follower_df

,UserName,Follower
0,Alice,Amy
1,Alice,Andrew
2,Alice,Benjamin
3,Alice,Daniel
4,Alice,Grace
...,...,...
115,Suezan,Charles
116,Suezan,Grace
117,Suezan,Jessica
118,Suezan,Nicole


In [63]:
Following_df

,UserName,Following,View,Hunny
0,Alice,Amy,96,55
1,Alice,Andrew,514,468
2,Alice,Benjamin,384,517
3,Alice,Charles,864,504
4,Alice,Grace,96,98
...,...,...,...,...
115,Suezan,Daniel,338,736
116,Suezan,Grace,833,199
117,Suezan,Jessica,407,575
118,Suezan,Kelly,84,922


In [64]:
Following_df['Score']=Following_df['View'] + (Following_df['Hunny']*2)
Following_df

,UserName,Following,View,Hunny,Score
0,Alice,Amy,96,55,206
1,Alice,Andrew,514,468,1450
2,Alice,Benjamin,384,517,1418
3,Alice,Charles,864,504,1872
4,Alice,Grace,96,98,292
...,...,...,...,...,...
115,Suezan,Daniel,338,736,1810
116,Suezan,Grace,833,199,1231
117,Suezan,Jessica,407,575,1557
118,Suezan,Kelly,84,922,1928


In [65]:
# 팔로잉 기준 Socre 정렬
Following_df2= Following_df.sort_values(by=['UserName','Score'], ascending=[True, False])
Following_df2

,UserName,Following,View,Hunny,Score
6,Alice,Kelly,696,954,2604
7,Alice,Nicole,205,906,2017
3,Alice,Charles,864,504,1872
1,Alice,Andrew,514,468,1450
2,Alice,Benjamin,384,517,1418
...,...,...,...,...,...
115,Suezan,Daniel,338,736,1810
117,Suezan,Jessica,407,575,1557
116,Suezan,Grace,833,199,1231
112,Suezan,Andrew,798,188,1174


In [66]:
# sort_values 뿌려주기
# 'Alice'기준 
Following_df2[Following_df2['UserName']=='Alice']['Following'].values

array(['Kelly', 'Nicole', 'Charles', 'Andrew', 'Benjamin', 'Sabina',
       'Jessica', 'Grace', 'Amy'], dtype=object)

# 유저간의 유사도 분석 -> 내가 친구 하지않은 친구 추천

## index = 'UserName', columns='Following'

In [67]:
title_user = Following_df.pivot_table('Score', index = 'UserName', columns='Following')
title_user

Following,Alice,Amy,Andrew,Benjamin,Charles,Daniel,Grace,Jessica,Kelly,Nicole,Sabina,Sharon,Suezan
UserName,,,,,,,,,,,,,
Alice,NaN,206.0,1450.0,1418.0,1872.0,NaN,292.0,477.0,2604.0,2017.0,1027.0,NaN,NaN
Amy,2273.0,NaN,NaN,NaN,1444.0,856.0,882.0,1872.0,1845.0,2473.0,503.0,NaN,NaN
Andrew,1061.0,405.0,NaN,459.0,1043.0,2171.0,815.0,1897.0,2874.0,1099.0,1672.0,NaN,NaN
Benjamin,2131.0,1638.0,1824.0,NaN,NaN,NaN,1226.0,1315.0,1568.0,1041.0,1012.0,1198.0,872.0
Charles,NaN,1887.0,1242.0,685.0,NaN,NaN,NaN,NaN,949.0,877.0,1210.0,1605.0,1252.0
Daniel,1838.0,2756.0,445.0,2270.0,NaN,NaN,584.0,1478.0,1537.0,1564.0,1684.0,NaN,NaN
Grace,2513.0,1171.0,1158.0,752.0,NaN,NaN,NaN,1885.0,1838.0,1296.0,2155.0,2686.0,597.0
Jessica,715.0,NaN,NaN,1349.0,772.0,1360.0,2134.0,NaN,NaN,NaN,2378.0,1965.0,633.0
Kelly,900.0,1160.0,1212.0,2147.0,1113.0,2108.0,896.0,1442.0,NaN,142.0,341.0,NaN,NaN


In [68]:
title_user.fillna(0, inplace=True)
title_user

Following,Alice,Amy,Andrew,Benjamin,Charles,Daniel,Grace,Jessica,Kelly,Nicole,Sabina,Sharon,Suezan
UserName,,,,,,,,,,,,,
Alice,0.0,206.0,1450.0,1418.0,1872.0,0.0,292.0,477.0,2604.0,2017.0,1027.0,0.0,0.0
Amy,2273.0,0.0,0.0,0.0,1444.0,856.0,882.0,1872.0,1845.0,2473.0,503.0,0.0,0.0
Andrew,1061.0,405.0,0.0,459.0,1043.0,2171.0,815.0,1897.0,2874.0,1099.0,1672.0,0.0,0.0
Benjamin,2131.0,1638.0,1824.0,0.0,0.0,0.0,1226.0,1315.0,1568.0,1041.0,1012.0,1198.0,872.0
Charles,0.0,1887.0,1242.0,685.0,0.0,0.0,0.0,0.0,949.0,877.0,1210.0,1605.0,1252.0
Daniel,1838.0,2756.0,445.0,2270.0,0.0,0.0,584.0,1478.0,1537.0,1564.0,1684.0,0.0,0.0
Grace,2513.0,1171.0,1158.0,752.0,0.0,0.0,0.0,1885.0,1838.0,1296.0,2155.0,2686.0,597.0
Jessica,715.0,0.0,0.0,1349.0,772.0,1360.0,2134.0,0.0,0.0,0.0,2378.0,1965.0,633.0
Kelly,900.0,1160.0,1212.0,2147.0,1113.0,2108.0,896.0,1442.0,0.0,142.0,341.0,0.0,0.0


In [69]:
user_based_collab = cosine_similarity(title_user,title_user)
user_based_collab = pd.DataFrame(user_based_collab, index = title_user.index, columns=title_user.index)

In [70]:
def Following_collab(Following):
    return user_based_collab[Following].sort_values(ascending=False)[:7]

In [71]:
Follower_list=list(Follower_df[Follower_df['UserName']=='Alice']['Follower'].values)

In [72]:
Following_collab_list=list(Following_collab("Alice").index)
Following_collab_list

['Alice', 'Suezan', 'Sabina', 'Andrew', 'Amy', 'Daniel', 'Nicole']

In [73]:
# 유저간의 유사도 분석 후 
# 나를 follower 사람 중에 추천
recommand_list=[]
for i in Following_collab_list :
    if i not in Follower_list :
        recommand_list.append(i)
recommand_list

['Alice', 'Sabina', 'Nicole']

In [74]:
# 내가 following 하지 않고, 나를 follower 사람 중에 추천
recommand_list[1:]

['Sabina', 'Nicole']

-----------

### 잠재요인 기반 협업필터링
- SVD
- 특정 사람과 비슷한 사람 추천 

In [75]:
# 친구 추천
Following_SVD = Following_df.pivot_table('Score', index = 'Following', columns='UserName')
Following_SVD

UserName,Alice,Amy,Andrew,Benjamin,Charles,Daniel,Grace,Jessica,Kelly,Nicole,Sabina,Sharon,Suezan
Following,,,,,,,,,,,,,
Alice,NaN,2273.0,1061.0,2131.0,NaN,1838.0,2513.0,715.0,900.0,NaN,NaN,1111.0,2644.0
Amy,206.0,NaN,405.0,1638.0,1887.0,2756.0,1171.0,NaN,1160.0,NaN,NaN,1411.0,309.0
Andrew,1450.0,NaN,NaN,1824.0,1242.0,445.0,1158.0,NaN,1212.0,1567.0,2825.0,NaN,1174.0
Benjamin,1418.0,NaN,459.0,NaN,685.0,2270.0,752.0,1349.0,2147.0,1859.0,947.0,NaN,2095.0
Charles,1872.0,1444.0,1043.0,NaN,NaN,NaN,NaN,772.0,1113.0,961.0,685.0,718.0,2403.0
Daniel,NaN,856.0,2171.0,NaN,NaN,NaN,NaN,1360.0,2108.0,1990.0,1478.0,306.0,1810.0
Grace,292.0,882.0,815.0,1226.0,NaN,584.0,NaN,2134.0,896.0,1541.0,2480.0,1567.0,1231.0
Jessica,477.0,1872.0,1897.0,1315.0,NaN,1478.0,1885.0,NaN,1442.0,309.0,1876.0,207.0,1557.0
Kelly,2604.0,1845.0,2874.0,1568.0,949.0,1537.0,1838.0,NaN,NaN,1912.0,2637.0,607.0,1928.0


In [76]:
Following_SVD.fillna(0, inplace=True)
Following_SVD

UserName,Alice,Amy,Andrew,Benjamin,Charles,Daniel,Grace,Jessica,Kelly,Nicole,Sabina,Sharon,Suezan
Following,,,,,,,,,,,,,
Alice,0.0,2273.0,1061.0,2131.0,0.0,1838.0,2513.0,715.0,900.0,0.0,0.0,1111.0,2644.0
Amy,206.0,0.0,405.0,1638.0,1887.0,2756.0,1171.0,0.0,1160.0,0.0,0.0,1411.0,309.0
Andrew,1450.0,0.0,0.0,1824.0,1242.0,445.0,1158.0,0.0,1212.0,1567.0,2825.0,0.0,1174.0
Benjamin,1418.0,0.0,459.0,0.0,685.0,2270.0,752.0,1349.0,2147.0,1859.0,947.0,0.0,2095.0
Charles,1872.0,1444.0,1043.0,0.0,0.0,0.0,0.0,772.0,1113.0,961.0,685.0,718.0,2403.0
Daniel,0.0,856.0,2171.0,0.0,0.0,0.0,0.0,1360.0,2108.0,1990.0,1478.0,306.0,1810.0
Grace,292.0,882.0,815.0,1226.0,0.0,584.0,0.0,2134.0,896.0,1541.0,2480.0,1567.0,1231.0
Jessica,477.0,1872.0,1897.0,1315.0,0.0,1478.0,1885.0,0.0,1442.0,309.0,1876.0,207.0,1557.0
Kelly,2604.0,1845.0,2874.0,1568.0,949.0,1537.0,1838.0,0.0,0.0,1912.0,2637.0,607.0,1928.0


In [77]:
# Following_SVD_T = Following_SVD?

In [78]:
SVD = TruncatedSVD(n_components=13)
matrixT=SVD.fit_transform(Following_SVD_T)
# SVD = TruncatedSVD(n_components=13)
# matrixT=SVD.fit_transform(Following_SVD)

In [79]:
matrixT

array([[ 3.54040218e+03,  5.56371949e+02,  8.57767403e+02,
        -1.09984318e+03,  3.02904883e+02,  1.89286769e+03,
         8.32299000e+02, -6.81419196e+02,  4.28373130e+01,
        -3.47906626e+02,  3.58467780e+02,  6.42077715e+01,
         2.16916543e+00],
       [ 3.84881087e+03, -2.89816922e+02,  2.24570597e+03,
        -4.35185839e+02, -1.07723356e+03, -6.66323047e+02,
        -2.08129324e+02, -2.76067459e+02,  3.54280062e+01,
         5.05018845e+01, -3.69131829e+02,  2.13280363e+02,
        -7.00016820e+00],
       [ 4.16208639e+03,  3.12681963e+02,  1.21999825e+03,
         3.02120949e+02, -9.77283015e+02, -8.54292942e+01,
         1.28438389e+03,  1.44910604e+03, -2.83329139e+02,
         1.01234465e+02,  1.43467922e+02, -6.42763402e+01,
        -6.10149521e+00],
       [ 3.89176639e+03, -1.36117106e+03, -7.49544757e+02,
        -9.21852293e+02,  1.29551979e+02, -7.69156196e+02,
        -9.54820215e+02,  2.35931256e+02, -1.71379750e+01,
         2.46344906e+02,  7.03292685e

In [80]:
# 피어슨 상관관계수 값
import numpy as np
corr = np.corrcoef(matrixT)
corr.shape

(13, 13)

In [81]:
Follower = Following_SVD.columns
Follower_list = list(Follower)
Follower_list

['Alice',
 'Amy',
 'Andrew',
 'Benjamin',
 'Charles',
 'Daniel',
 'Grace',
 'Jessica',
 'Kelly',
 'Nicole',
 'Sabina',
 'Sharon',
 'Suezan']

In [82]:
Following_SVD.columns

Index(['Alice', 'Amy', 'Andrew', 'Benjamin', 'Charles', 'Daniel', 'Grace',
       'Jessica', 'Kelly', 'Nicole', 'Sabina', 'Sharon', 'Suezan'],
      dtype='object', name='UserName')

In [83]:
def recommand_Following(following):
    target = Follower_list.index(following)
    corr_target = corr[target]
    result = list(Follower[(corr_target>=0.7)])[:7]
    return result

In [84]:
recommand_Following('Alice')

['Alice', 'Sabina', 'Suezan']

======================================================================================